<a href="https://colab.research.google.com/github/Kevin2558/Data_Science/blob/main/03_Transformaciones/Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Autoencoders

# Como funciona:

# Es un tipo de agrupacion similar a PCA, que lo que hace es reducir la
# dimensionalidad. Tiene la ventaja que se puede decodificar y volver a los
# datos originales y de que puede capturar relaciones no lineales. Autoencoders
# son redes neuronales entrenadas para reconstruir sus entradas. Aprenden una
# representacion comprimida (latente).

# Su estrutura viene dada por:

# Encoder: Proyecta a espacio latente (reducir la dimensionalidad)
# Decoder: Reconstruye la entrada original desde el espacio latente

# Elige los pesos de la red neuronal a través de minimizar el error que ocurre
# en la reconstruccion de los datos.

# Aunque este no es un proceso supervisado, cuando se construye la red hay un
# entrenamiento para encontrar los pesos y armar los encoders y decoders.

# Tipos de autoencoders:

# Clasico: Capas totalmente conectadas, activaciones no lineales.
# Convolucional: Usa convoluciones, ideal para imagenes.
# Denoising: Entrena para reconstruir desde entradas corruptas

# encoding_dim Dimension del espacio latente
# acivation Funciones de activacion
# loss Metrica de reconstruccion
# optimizer Algoritmo de optimizacion
# epochs Numero de epocas de entrenamiento

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

In [3]:
wine = load_wine()
X = wine.data
y = wine.target

In [5]:
# Preprocesamiento
X_scaled = StandardScaler().fit_transform(X)

In [6]:
X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)

In [7]:
X.shape

(178, 13)

In [8]:
# Definir arquitectura del autoencoder
input_dim = X.shape[1]
encoding_dim = 2

In [11]:
# Capa inicial
input_layer = Input(shape=(input_dim,))
# Primera capa de la red
encoded = Dense(8, activation='relu')(input_layer)
# Capa intermedia
encoded = Dense(input_dim, activation='relu')(encoded)

In [13]:
# Primera capa de decomprension
decoded = Dense(8, activation='relu')(encoded)
# Capa final
decoded = Dense(input_dim, activation='linear')(decoded)

In [14]:
# Modelo
autoencoder = Model(inputs=input_layer, outputs=decoded)
encoder = Model(inputs=input_layer, outputs=encoded)

In [ ]:
# Compilar y entrenar
autoencoder.compile(optimizer=Adam(learning_rate=0.01), loss=MeanSquaredError())
history = autoencoder.fit(X_train, X_train,
                          validation_data=(X_test, X_test),
                          epochs=100,
                          batch_size=16,
                          verbose=0)